In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
def Supertrend(df, atr_period=10, multiplier=3):

    high, low, close = df['High'], df['Low'], df['Adj Close']

    true_range = pd.concat([high - low, high - close.shift(), close.shift() - low], axis=1).abs().max(axis=1)
    average_true_range = true_range.ewm(alpha=1/atr_period, min_periods=atr_period).mean()
    df['ATR'] = average_true_range

    hl2 = (high + low) / 2

    upper_band = hl2 + (multiplier * average_true_range)
    lower_band = hl2 - (multiplier * average_true_range)

    supertrend = pd.Series(True, index=df.index)

    for i in range(1, len(df.index)):
        if close.iloc[i] > upper_band.iloc[i-1]:
            supertrend.iloc[i] = True

        elif close.iloc[i] < lower_band.iloc[i-1]:
            supertrend.iloc[i] = False

        else:
            supertrend.iloc[i] = supertrend.iloc[i-1]

            # When the band is below prices, it only moves upwards or sideways, never downwards;
            # and when the band is above prices, it only moves downwards or sideways, never upwards.
            if supertrend.iloc[i] and lower_band.iloc[i] < lower_band.iloc[i-1]:
                lower_band.iloc[i] = lower_band.iloc[i-1]
            if not supertrend.iloc[i] and upper_band.iloc[i] > upper_band.iloc[i-1]:
                upper_band.iloc[i] = upper_band.iloc[i-1]

        upper_band.iloc[i] = np.nan if supertrend.iloc[i] else upper_band.iloc[i]
        lower_band.iloc[i] = np.nan if not supertrend.iloc[i] else lower_band.iloc[i]

    result_df = pd.DataFrame({'Supertrend': supertrend, 'Lowerband': lower_band, 'Upperband': upper_band}, index=df.index)

    return result_df

symbol = 'PGSUS.IS'
df = yf.download(symbol, start='2022-01-01')

supertrend = Supertrend(df)
df = df.join(supertrend)

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(2, 1, 1)
plt.plot(df['Adj Close'], label='Close Price')
plt.plot(df['Lowerband'], color='green', label='Lowerband')
plt.plot(df['Upperband'], color='red', label='Upperband')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(df['ATR'], color='purple', label='ATR')
plt.legend()

plt.suptitle(f'Supertrend Analysis for {symbol} (For Educational Purposes Only)', fontsize=16, y=0.95)

plt.show()

In [ ]:
def backtest_supertrend(df, initial_investment=100000, trading_commission=0):
    is_uptrend = df['Supertrend']
    close = df['Adj Close']

    position = False
    total_equity = initial_investment
    shares = 0
    entry_points = []
    exit_points = []

    for i in range(2, len(df)):

        if not position and is_uptrend.iloc[i]:
            shares_float = total_equity / close.iloc[i]
            shares_rounded = math.floor(shares_float)
            shares = shares_rounded
            total_equity -= shares * close.iloc[i]
            entry_points.append((i, close.iloc[i]))
            position = True
            print(f'Buy {shares} shares at {round(close.iloc[i],2)} on {df.index[i].strftime("%d/%m/%Y")}')

        elif position and not is_uptrend.iloc[i]:
            total_equity += shares * close.iloc[i] - trading_commission
            exit_points.append((i, close.iloc[i]))
            position = False
            print(f'Sell at {round(close.iloc[i],2)} on {df.index[i].strftime("%d/%m/%Y")}')

    total_equity += shares * close.iloc[i] - trading_commission if position else 0

    earning = round(total_equity - initial_investment, 2)
    roi = round(earning / initial_investment*100, 1)
    print("*" * 30)
    print(f'Total profit after investing {initial_investment}₺ is {round(earning,2)}₺\nReturn on Investment (ROI): %{roi}')
    print("*" * 30)
    return entry_points, exit_points, total_equity

entry_points, exit_points, roi = backtest_supertrend(df)